# < 프로젝트 목표>
1. 출시 직전과 출시 직후의 V30과 Note8의 SNS(twitter)를 크롤링 하여 자연어 처리를 통해 해당 제품의 기대 요인을 분석 해 본다
2. 분석과정에서 MongoDB와 MySQL을 연동한다
3. 다른 자연어 데이터와는 다르게 트위터는 **특수 문자, 문법 파괴가 심하고 신조어의 빈도가 많다.** 해당 특징을 감안하여 자연어 분석을 진행한다 

In [2]:
import requests
import json
import re
from bs4 import BeautifulSoup
from html.parser import HTMLParser

In [3]:
# 모델링 시간 구하기
from time import time

def check_time(t0):
    dur = time() - t0
    print ("계산에 걸린시간 : {:.3f} 초".format(dur))

# 1. 크롤링

### 1) Crawling Function

In [26]:
# 반복 함수, generator 사용
def generate(model_name, n, url, cookie):
    print (" \n {} 에 관하여 크롤링 중입니다.".format(model_name))
    
    
    i = 0
    
    t0 = time() # 시간 초기화
    while i < n: # 주어진 수만큼 반복
        url_phone = url.format(i)
        key, value = sns_crawler(url_phone, cookie)
        
        yield key, value
        # key(작성자)의 중복을 막기 위해 dict의 key는 바로 작성자로 넘긴다
        i += 1
    
    check_time(t0)
    
    

In [5]:
# 크롤링 객체 생성 및 내용 가져오는 함수

def sns_crawler(url, cookie):
    header_phone = {}
    header_phone['cookie'] = cookie
    res_phone = requests.get(url, headers = header_phone)
    # url 만으로는 크롤링이 되지 않아 header정보 추가
    data_phone = json.loads(res_phone.content)
    

    contents_dict = {}

    
        
    value_templist = [] # dict의 value가 들어갈 list
    # key
    contents_key = data_phone['RESULT']['SOCIALWEB_BOARD_V2']['r']['ds']['data']['user_screen_name']

    # value
    temp_value1 = data_phone['RESULT']['SOCIALWEB_BOARD_V2']['r']['ds']['data']['pub_date'] 
    temp_value2 = data_phone['RESULT']['SOCIALWEB_BOARD_V2']['r']['ds']['data']['text'] 

    value_templist.append(temp_value1)
    value_templist.append(temp_value2)
        

    return contents_key, value_templist

### 2) Making dictionary

In [28]:


def make_snsdict (model_name, iter_num, url, cookie):
    
    gen = generate(model_name, iter_num, url, cookie)

    #중복불가 crawling
    dict_nondup = {}
    dict_dup= {}
    key_num = 0
    
    for i in gen:
        dict_nondup[i[0]] = i[1]         
        dict_dup[key_num] = [i[0], i[1]]
        key_num += 1
    print ("\n 1) 모델: {} \n 2) 중복제거 contents 개수는 {} 입니다.".format(model_name, len(dict_nondup)))
    
    
    #gen = generate(model_name, iter_num, url, cookie)
    
    #중복허용 crawling
    #dict_dup= {}
    #key_num = 0
    #for i in gen:
     #   dict_dup[key_num] = [i[0], i[1]]
      #  key_num += 1
    print ("\n 1) 모델 : {} \n 2) 중복허용 contents 개수는 {} 입니다.".format(model_name, len(dict_dup)))
    
    return dict_nondup, dict_dup



### 3) V30

* URL & Cookie정보

In [16]:
# url
url_v30 = 'https://search.daum.net/qsearch?w=swBoard&search_option=tab\
&sort_type=socialweb&search_type=&st_query=&user=&begindate=&enddate=\
&uk=WcD2JdPnaUwAAC5J8EIAAACf&mk=Wefzi8uFookAAAqUzfEAAAAe&n=1&p={}\
&viewtype=json&q=v30'

#cookie
cookie_v30 = 'uvkey=WcD2JdPnaUwAAC5J8EIAAACf; SLEVEL=1; ssab=; \
ODT=NNSZ_SNEZ_BR1Z_WSAZ_IIMZ_FGKZ_VO2Z_; \
DDT=NKSZ_IVRZ_CCBZ_1DVZ_DICZ_GG1Z_MS2Z_; DTQUERY=v30; \
TIARA=debDoszx9N6z8uscxFx8jN1SPBs.iMogrdFhye9t3Q5.\
s4FFRLTkMZe-HSVbqtiKAXL9lcS39IhPohGj1ZKWGC6x7EZ6I99s'

* generator 객체 생성 및 호출


In [32]:
dict_nondup_v30, dict_dup_v30 = make_snsdict('v30', 20000, url_v30, cookie_v30)

 
 v30 에 관하여 크롤링 중입니다.
계산에 걸린시간 : 2277.015 초

 1) 모델: v30 
 2) 중복제거 contents 개수는 369 입니다.

 1) 모델 : v30 
 2) 중복허용 contents 개수는 20000 입니다.


### 4) Note8

* URL & Cookie 정보

In [33]:
# url
url_note8 = 'https://search.daum.net/qsearch?w=swBoard&search_option=tab&\
sort_type=&search_type=&st_query=&user=&uk=WegTisuFolkAAGZHIwoAAACL&\
mk=WegTpdPnaW0AABbBK08AAACU&begindate=&enddate=20171019112427&\
retweet=&q=%EB%85%B8%ED%8A%B88&viewtype=json&p={}&n=1'

#cookie
cookie_note8 = 'uvkey=WegTisuFolkAAGZHIwoAAACL; ssab=; \
ODT=IIMZ_VO2Z_NNSZ_BR1Z_CCBZ_FGKZ_WSAZ_; \
DDT=NKSZ_IVRZ_DICZ_GG1Z_1DVZ_SNPZ_MS2Z_; DTQUERY=%EB%85%B8%ED%8A%B88;\
TIARA=8A51vhNbAeorPdcuTVLyR8R8bhlVhNO_wVuBsKod5Fzk8GJ54w8hP5ftiAXLR\
PYzThjfJmjS4Yr.bH6se98rGw00'

In [34]:
dict_nondup_note8, dict_dup_note8 = make_snsdict('노트8', 20000, url_note8, cookie_note8)

 
 노트8 에 관하여 크롤링 중입니다.
계산에 걸린시간 : 2259.747 초

 1) 모델: 노트8 
 2) 중복제거 contents 개수는 707 입니다.

 1) 모델 : 노트8 
 2) 중복허용 contents 개수는 20000 입니다.


In [ ]:
# pymongo에 넣을 때 key 값이 int 면 오류  -> str 로 type 변경
i = 0
for i in range(20000):
    key_str = str(i)
    dict_dup_v30[key_str] = dict_dup_v30.pop(i)
    i += 1

In [44]:
# pymongo에 넣을 때 key 값이 int 면 오류  -> str 로 type 변경
i = 0
for i in range(20000):
    key_str = str(i)
    dict_dup_note8[key_str] = dict_dup_note8.pop(i)
    i += 1

In [180]:
import pandas as pd

pd.DataFrame(dict_nondup_note8).T.head()

,0,1
02_youth,2017-10-15 18:14:28,엄마가 요금제 무제한해주고 폰 <b>노트</b> <b>8</b>로 바꿔줬다 ㅎㅎ 월...
0_Kio,2017-10-18 00:05:24,S8이랑 <b>노트</b><b>8</b>은 사실상 월 5처넌 차이 정도 밖에 안나는...
0nicolessong0,2017-10-16 20:34:05,혹시 <b>노트</b><b>8</b> 사전 예약 하신분들 중 문자 받으신 분ㅜㅠ 나...
10PVlaWXJWUmn90,2017-10-18 07:47:05,미래의 <b>노트</b>님하고 론님도 집에 빨리들어가게 해주세요... <b>8</b...
10_cc6121,2017-10-19 11:11:51,허루ㅜㅜㅜㅜ<b>노트</b><b>8</b>케이스드디어이뿡고샄댜ㅠㅜㅠㅠㅠㅠㅠㅠ


In [181]:
pd.DataFrame(dict_dup_note8).T.head()

,0,1
0,xlsx464,"[2017-10-19 11:11:56, 후... <b>노트</b><b>8</b> 계..."
1,xlsx464,"[2017-10-19 11:11:56, 후... <b>노트</b><b>8</b> 계..."
10,U3U_0212,"[2017-10-19 10:15:34, 꿈속에서 폰 <b>노트</b><b>8</b>..."
100,WhitestoneHodoo,"[2017-10-18 21:38:45, 갤럭시<b>노트</b><b>8</b> 리뷰 ..."
1000,indigo_93,"[2017-10-14 22:33:09, @_sailer_ <b>노트</b><b>8<..."


In [182]:
pd.DataFrame(dict_nondup_v30).T.head()

,0,1
04140423,2017-10-07 11:27:37,<b>V30</b>블락비 짱짱맨 마이존사랑해요ㅠㅠ [<b>V30</b> X bloc...
100uky,2017-09-01 21:09:40,"이번에 꽤 잘나왔던데,,,떨어졌구나. it review 자존심 바닥이구나! 안녕 L..."
1111111kChoi,2017-10-12 19:17:49,카메라 촬영 하드웨어는 잘 만든듯하다. 그러나 촬영항 영상을 쉽게 스마트폰앱으로 편...
1dayofgirl,2017-09-06 05:15:54,[550x] https://t.co/fRvosNySQk [450x] https://...
62wavesplayball,2017-09-01 21:20:23,사랑해요 엘지 #사랑해요 #엘지 #금성 #골드스타 #체험단 #항상탈락 #<b>V3...


In [183]:
pd.DataFrame(dict_dup_v30).T.head()

,0,1
0,yamadaikenbot,"[2017-10-19 06:10:48, 야마가타 대학의 강점은 올레드, 나스카 지상..."
1,yamadaikenbot,"[2017-10-19 06:10:48, 야마가타 대학의 강점은 올레드, 나스카 지상..."
10,raycatnet,"[2017-10-13 10:12:35, lg <b>v30</b> 화면 캡처 방법과 ..."
100,YDGStar,"[2017-10-17 08:26:37, ■갤럭시노트8,<b>V30</b>,아이폰7,..."
1000,tnqls119,"[2017-09-27 10:23:32, [TF현장] `갤노트8 40만 원, <b>V..."


# 2. Insert Data to MongoDB
* 리트윗이나 광고글의 목적으로 같은 ID가 같은 내용을 여러번 올리는 경우도 있고, 다른 내용을 올리는 경우도 있다
* non_dup : 중복을 허용하지 않기 위해 dictionary의 key값을 id로 하여, value는 최종 트윗 내용으로 대체한다
* dup : 추후 분석에 필요할 수도 있어 id와 text를 같은 내용이라도 전부 저장한다. 
* 형식이 유동적이어서 일단 크롤링한 내용을 mongodb에 저장한다

### 1) 형식에 맞게 key값을 id, date, text 로 하는 함수

In [170]:
# 중복허용 하지 않은 결과
def make_dbcontents_nondup(contents_dict):
    t0 = time()
    
    total_list = []
    keys = list(map(lambda x: {'id':x} , contents_dict.keys()))
    i = 0
    for value in contents_dict.values():
        
        total_dict = {}
        total_dict['id'] = keys[i]['id']
        total_dict['date'] = list(contents_dict.values())[:][i][0]
        total_dict['text'] = list(contents_dict.values())[:][i][1]
        i += 1
        total_list.append(total_dict)
        
    check_time(t0)
        
    return total_list

In [171]:
# 중복 허용 한 결과
def make_dbcontents_dup(contents_dict):
    t0 = time()
     
    total_list = []
    i = 0
    for value in contents_dict.values():
        
        total_dict = {}
        total_dict['id'] = list(contents_dict.values())[:][:][i][0]
        total_dict['date'] = list(contents_dict.values())[:][:][:][i][1][0]
        total_dict['text'] = list(contents_dict.values())[:][:][:][i][1][1]
        i += 1
        total_list.append(total_dict)
        
    check_time(t0)
        
    return total_list

In [91]:
import pymongo 

mongo_server = '52.14.175.152'
mongo_port = 54329

from pymongo import MongoClient

mongo = MongoClient(mongo_server, mongo_port)


In [116]:
db_note8_nondup = mongo.sns.note8_nondup
db_v30_nondup = mongo.sns.v30_nondup

db_note8_dup = mongo.sns.note8_dup
db_v30_dup = mongo.sns.v30_dup

### 2) V30

In [172]:
list_nondup_v30 = make_dbcontents_nondup(dict_nondup_v30)
list_dup_v30 = make_dbcontents_dup(dict_dup_v30)



계산에 걸린시간 : 0.005 초
계산에 걸린시간 : 30.676 초


In [175]:
# insert to mongodb
db_v30_nondup.insert_many(list_nondup_v30)
db_v30_dup.insert_many(list_dup_v30)


### 3) Note8

In [176]:
list_nondup_note8 = make_dbcontents_nondup(dict_nondup_note8)
list_dup_note8 = make_dbcontents_dup(dict_dup_note8)


계산에 걸린시간 : 0.010 초
계산에 걸린시간 : 30.325 초


In [177]:
# insert to mongodb
db_note8_nondup.insert_many(list_nondup_note8)
db_note8_dup.insert_many(list_dup_note8)